## LIBRARIES

In [21]:
import os
import pyspark.sql.functions as F

In [22]:
os.listdir('/home/daman/Downloads/financial_trans')

['users_data.csv',
 'transactions_data.csv',
 'mcc_codes.json',
 'train_fraud_labels.json',
 'cards_data.csv']

## SPARK SESSION

In [23]:
from pyspark.sql import SparkSession
spark=(SparkSession
        .builder
        .appName("SparkApp")
        .getOrCreate())


In [24]:
spark

## RAW_USERS

In [25]:
df1=spark.read.csv("/home/daman/Downloads/financial_trans/users_data.csv",header=True)
df1.createOrReplaceTempView('users_data')

df1.limit(1).show()


+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
| id|current_age|retirement_age|birth_year|birth_month|gender|      address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|825|         53|            66|      1966|         11|Female|462 Rose Lane|   34.15|  -117.76|           $29278|       $59696|   $127613|         787|               5|
+---+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+



In [26]:
df1.write.mode("overwrite").parquet("./output/raw_users")

In [27]:
with open ("/home/daman/Downloads/notebooks/financial-task/query/raw_users.sql") as f:
    users=f.read()

In [28]:

spark.sql(users).toPandas()
# spark.sql("SELECT * FROM RAW_USERS").toPandas().head(2)


,id,gender,birth_date,address,yearly_income,total_debt,credit_score,credit_rating,num_credit_cards,retirement_age,years_for_retirement
0,825,Female,1966-11-01,462 Rose Lane,59696.000,127613.000,787,4,5,66,7
1,1746,Female,1966-12-01,3606 Federal Boulevard,77254.000,191349.000,701,3,5,68,9
2,1718,Female,1938-11-01,766 Third Drive,33483.000,196.000,698,3,5,67,-20
3,708,Female,1957-01-01,3 Madison Street,249925.000,202328.000,722,3,4,63,-5
4,1164,Male,1976-09-01,9620 Valley Stream Drive,109687.000,183855.000,675,3,1,70,21
...,...,...,...,...,...,...,...,...,...,...,...
1995,986,Male,1987-07-01,6577 Lexington Lane,48010.000,87837.000,703,3,3,70,32
1996,1944,Female,1957-11-01,2 Elm Drive,49378.000,104480.000,740,4,4,65,-3
1997,185,Female,1973-01-01,276 Fifth Boulevard,30942.000,71066.000,779,4,3,67,15
1998,1007,Male,1954-02-01,259 Valley Boulevard,54654.000,27241.000,618,2,1,60,-11


<!-- RAW_CARDS -->

## RAW_CARDS

In [29]:
df2=spark.read.csv("/home/daman/Downloads/financial_trans/cards_data.csv",header=True)
df2.createOrReplaceTempView('cards_data')

#df2.limit(1).show()
df2.toPandas().head(1)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No


In [30]:
df2.write.mode("overwrite").parquet("./output/raw_cards")

In [31]:
with open("/home/daman/Downloads/notebooks/financial-task/query/raw_cards.sql")as f :
    cards=f.read()

In [32]:
spark.sql(cards).show()
# spark.sql("SELECT * FROM RAW_CARDS").toPandas().head(2)

+----+---------+----------+---------------+------------+
|  id|client_id|card_brand|      card_type|credit_limit|
+----+---------+----------+---------------+------------+
|4524|      825|      Visa|          Debit|   24295.000|
|2731|      825|      Visa|          Debit|   21968.000|
|3701|      825|      Visa|          Debit|   46414.000|
|  42|      825|      Visa|         Credit|   12400.000|
|4659|      825|Mastercard|Debit (Prepaid)|      28.000|
|4537|     1746|      Visa|         Credit|   27500.000|
|1278|     1746|      Visa|          Debit|   28508.000|
|3687|     1746|Mastercard|          Debit|    9022.000|
|3465|     1746|Mastercard|Debit (Prepaid)|      54.000|
|3754|     1746|Mastercard|Debit (Prepaid)|      99.000|
|5144|     1718|Mastercard|          Debit|   31599.000|
|2029|     1718|Mastercard|          Debit|   27480.000|
|2379|     1718|Mastercard|          Debit|   26743.000|
|2732|     1718|      Visa|          Debit|   31463.000|
|4706|     1718|Mastercard|    

## RAW_MERCHANTS

In [33]:
import json


with open("/home/daman/Downloads/financial_trans/mcc_codes.json", "r") as f:
    raw_dict = json.load(f)


data = [{"mcc": k, "category": v} for k, v in raw_dict.items()]


df_mcc = spark.createDataFrame(data)
df_mcc.createOrReplaceTempView('mcc_codes')
df_mcc.show(10)



+--------------------+----+
|            category| mcc|
+--------------------+----+
|Eating Places and...|5812|
|    Service Stations|5541|
|Amusement Parks, ...|7996|
|Grocery Stores, S...|5411|
|Tolls and Bridge ...|4784|
|Utilities - Elect...|4900|
|         Book Stores|5942|
|Fast Food Restaur...|5814|
|      Money Transfer|4829|
|   Department Stores|5311|
+--------------------+----+
only showing top 10 rows



In [34]:
df_mcc.write.mode("overwrite").parquet("./output/raw_merchants")

In [35]:
with open("/home/daman/Downloads/notebooks/financial-task/query/raw_merchants.sql", "r") as f:
    merchants=f.read()

In [ ]:
spark.sql(merchants).show()
# run below cell for transaction_data and then this.
# spark.sql('SELECT * FROM RAW_MERCHANTS').show(10, truncate=False)

+-----------+--------------+--------------+-----+--------------------+
|merchant_id| merchant_city|merchant_state|  zip|            category|
+-----------+--------------+--------------+-----+--------------------+
|      64566|        Mobile|            AL|36693|Accounting, Audit...|
|      95620|     Hampstead|            MD|21074|Insurance Sales, ...|
|     100331|       De Land|            IL|61839|Insurance Sales, ...|
|      78556|  Cochranville|            PA|19330|Insurance Sales, ...|
|      76639|    Willoughby|            OH|44094|  Electronics Stores|
|      39821|        Bethel|            NC|27812|Computers, Comput...|
|      15853|     Thorndale|            PA|19372|Lumber and Buildi...|
|      68002|       Oakland|            CA|94612|Lumber and Buildi...|
|      26909|    Ravensdale|            WA|98051|Lumber and Buildi...|
|      26909|  Indianapolis|            IN|46217|Lumber and Buildi...|
|      26909|         Dover|            PA|17315|Lumber and Buildi...|
|     

## RAW_TRANSACTIONS

In [38]:
df3=spark.read.csv("/home/daman/Downloads/financial_trans/transactions_data.csv",header=True)
df3.createOrReplaceTempView('transactions_data')

df3.limit(10).show()
# df3.toPandas().head(1)

+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00| Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57| Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00| Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00| Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|      848|   3915| 

In [40]:
with open("/home/daman/Downloads/notebooks/financial-task/query/raw_transactions.sql", "r") as f:
    querry=f.read()

In [41]:
df3.write.mode("overwrite").parquet("./output/raw_transactions")

In [42]:
spark.sql(querry ).show()
# spark.sql("SELECT * FROM RAW_TRANSACTIONS").show(10,truncate=False)

+-------+-------------------+---------+-------+-----------+-------+------------------+
|     id|               date|client_id|card_id|merchant_id| amount|          use_chip|
+-------+-------------------+---------+-------+-----------+-------+------------------+
|7475327|2010-01-01 00:01:00|     1556|   2972|      59935|-77.000| Swipe Transaction|
|7475328|2010-01-01 00:02:00|      561|   4575|      67570| 14.570| Swipe Transaction|
|7475329|2010-01-01 00:02:00|     1129|    102|      27092| 80.000| Swipe Transaction|
|7475331|2010-01-01 00:05:00|      430|   2860|      27092|200.000| Swipe Transaction|
|7475332|2010-01-01 00:06:00|      848|   3915|      13051| 46.410| Swipe Transaction|
|7475333|2010-01-01 00:07:00|     1807|    165|      20519|  4.810| Swipe Transaction|
|7475334|2010-01-01 00:09:00|     1556|   2972|      59935| 77.000| Swipe Transaction|
|7475335|2010-01-01 00:14:00|     1684|   2140|      39021| 26.460|Online Transaction|
|7475336|2010-01-01 00:21:00|      335|   5

# Test

In [ ]:
spark.sql('''
select distinct
t.merchant_id  ,
t.merchant_city,
t.merchant_state,
cast(cast(t.zip as int) as string) zip,
m.mcc,
m.category 
from transactions_data t 
left join mcc_codes m
on t.mcc=m.mcc
          where t.merchant_id = 73025
                     
''').show(10, truncate=False)

+-----------+-------------+--------------+-----+----+----------------------------------------------+
|merchant_id|merchant_city|merchant_state|zip  |mcc |category                                      |
+-----------+-------------+--------------+-----+----+----------------------------------------------+
|73025      |Morrisville  |NC            |27560|8931|Accounting, Auditing, and Bookkeeping Services|
+-----------+-------------+--------------+-----+----+----------------------------------------------+



In [ ]:
spark.read.parquet('./spark-warehouse/raw_merchants').show(1, truncate=False)

+-----------+-------------+--------------+-----+----------------+
|merchant_id|merchant_city|merchant_state|zip  |category        |
+-----------+-------------+--------------+-----+----------------+
|26810      |Plattekill   |NY            |12568|Service Stations|
+-----------+-------------+--------------+-----+----------------+
only showing top 1 row



In [ ]:
spark.read.parquet('./output/raw_users').toPandas()


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,6577 Lexington Lane,40.65,-73.58,$23550,$48010,$87837,703,3
1996,1944,62,65,1957,11,Female,2 Elm Drive,38.95,-84.54,$24218,$49378,$104480,740,4
1997,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,$15175,$30942,$71066,779,3
1998,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,$25336,$54654,$27241,618,1
